# Q1: Comprehensive EDA - Phân Tích Dữ Liệu Chi Tiết

Notebook này trả lời đầy đủ các yêu cầu:
1. Kiểm tra khoảng thời gian và tần suất
2. Tính tỷ lệ thiếu theo biến và theo thời gian
3. Boxplot/quantile để phát hiện outliers
4. Vẽ chuỗi PM2.5 (toàn bộ + zoom 1-2 tháng)
5. Kiểm tra tự tương quan (lag 24h, 168h)
6. Kiểm tra tính dừng (ADF/KPSS)
7. Giải thích biến thiếu đáng lo nhất

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Load Data

In [ ]:
from src.classification_library import load_beijing_air_quality, clean_air_quality_df

RAW_ZIP_PATH = '../data/raw/PRSA2017_Data_20130301-20170228.zip'

# Load raw data
df_raw = load_beijing_air_quality(use_ucimlrepo=False, raw_zip_path=RAW_ZIP_PATH)
print(f'Raw data shape: {df_raw.shape}')
print(f'Columns: {list(df_raw.columns)}')
df_raw.head()

In [ ]:
# Clean data
df = clean_air_quality_df(df_raw)
print(f'Cleaned data shape: {df.shape}')
df.head()

## 2. Kiểm tra khoảng thời gian và tần suất

**Mục tiêu:** Xác định start/end date và kiểm tra tính liên tục của dữ liệu theo giờ

In [ ]:
# Chọn một trạm để phân tích chi tiết
STATION = 'Aotizhongxin'
df_station = df[df['station'] == STATION].copy()
df_station = df_station.sort_values('datetime').reset_index(drop=True)

print(f"=" * 60)
print(f"THÔNG TIN KHOẢNG THỜI GIAN DỮ LIỆU - Trạm {STATION}")
print(f"=" * 60)
print(f"Start date: {df_station['datetime'].min()}")
print(f"End date  : {df_station['datetime'].max()}")
print(f"Duration  : {(df_station['datetime'].max() - df_station['datetime'].min()).days} days")
print(f"Total rows: {len(df_station):,}")

# Tính số giờ dự kiến
expected_hours = (df_station['datetime'].max() - df_station['datetime'].min()).total_seconds() / 3600 + 1
print(f"Expected hourly records: {expected_hours:,.0f}")
print(f"Actual records: {len(df_station):,}")
print(f"Coverage: {len(df_station)/expected_hours*100:.2f}%")

# Kiểm tra tần suất
time_diffs = df_station['datetime'].diff().dropna()
print(f"\nTẦN SUẤT THỜI GIAN:")
print(f"Mode (most common): {time_diffs.mode()[0]}")
print(f"Median: {time_diffs.median()}")
print(f"Non-1hour gaps: {(time_diffs != pd.Timedelta(hours=1)).sum()} ({(time_diffs != pd.Timedelta(hours=1)).sum()/len(time_diffs)*100:.2f}%)")

## 3. Tỷ lệ thiếu theo từng biến

**Mục tiêu:** Tính missing rate cho mỗi biến và phân tích missing pattern

In [ ]:
print("=" * 60)
print("TỶ LỆ THIẾU THEO TỪNG BIẾN")
print("=" * 60)

missing_stats = pd.DataFrame({
    'Missing_Count': df_station.isna().sum(),
    'Missing_Percent': df_station.isna().mean() * 100,
    'Data_Type': df_station.dtypes
}).sort_values('Missing_Percent', ascending=False)

print(missing_stats)

# Visualize missing percentages
fig, ax = plt.subplots(figsize=(12, 6))
missing_stats['Missing_Percent'].plot(kind='barh', ax=ax, color='coral')
ax.set_xlabel('Missing Percentage (%)', fontsize=12)
ax.set_title(f'Tỷ lệ thiếu theo biến - Trạm {STATION}', fontsize=14, fontweight='bold')
ax.axvline(x=10, color='red', linestyle='--', alpha=0.5, label='10% threshold')
ax.legend()
plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
high_missing = missing_stats[missing_stats['Missing_Percent'] > 10]
if len(high_missing) > 0:
    print(f"Các biến có tỷ lệ thiếu > 10%: {list(high_missing.index)}")
else:
    print("Không có biến nào có tỷ lệ thiếu > 10%")

## 4. Missing Pattern theo thời gian

**Mục tiêu:** Quan sát xem dữ liệu thiếu có tập trung ở giai đoạn nào hay rải đều

In [ ]:
# Tạo missing indicator cho PM2.5
df_station['PM25_missing'] = df_station['PM2.5'].isna().astype(int)

# Tính tỷ lệ thiếu theo tháng
df_station['year_month'] = df_station['datetime'].dt.to_period('M')
monthly_missing = df_station.groupby('year_month')['PM25_missing'].agg(['mean', 'sum', 'count'])
monthly_missing['percent'] = monthly_missing['mean'] * 100

print("TỶ LỆ THIẾU PM2.5 THEO THÁNG:")
print(monthly_missing.sort_values('percent', ascending=False).head(10))

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# Plot 1: Missing pattern over time
ax1 = axes[0]
monthly_missing['percent'].plot(ax=ax1, color='red', linewidth=2)
ax1.set_ylabel('Missing %', fontsize=11)
ax1.set_title('Tỷ lệ thiếu PM2.5 theo tháng', fontsize=13, fontweight='bold')
ax1.axhline(y=monthly_missing['percent'].mean(), color='blue', linestyle='--', 
            label=f'Mean: {monthly_missing["percent"].mean():.2f}%')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Missing count over time (daily resolution)
ax2 = axes[1]
df_station['date'] = df_station['datetime'].dt.date
daily_missing = df_station.groupby('date')['PM25_missing'].sum()
daily_missing.plot(ax=ax2, color='orange', alpha=0.7)
ax2.set_ylabel('Missing records per day', fontsize=11)
ax2.set_title('Số lượng giờ thiếu PM2.5 mỗi ngày', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
if monthly_missing['percent'].std() > 5:
    print("❌ Dữ liệu thiếu KHÔNG RẢI ĐỀU - có giai đoạn thiếu tập trung")
    peak_missing = monthly_missing.sort_values('percent', ascending=False).head(3)
    print(f"Các tháng thiếu nhiều nhất: {list(peak_missing.index)}")
else:
    print("✅ Dữ liệu thiếu tương đối RẢI ĐỀU theo thời gian")

## 5. Boxplot và Quantile - Phát hiện Outliers

**Mục tiêu:** Sử dụng boxplot và phân tích quantile để nhận diện ngoại lai và phân phối lệch

In [ ]:
# Chọn các biến số quan trọng
numeric_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
numeric_cols = [col for col in numeric_cols if col in df_station.columns]

print("=" * 60)
print("PHÂN TÍCH QUANTILE CHO CÁC BIẾN")
print("=" * 60)

quantile_stats = df_station[numeric_cols].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
print(quantile_stats)

# Boxplots for air quality variables
air_quality_vars = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
air_quality_vars = [col for col in air_quality_vars if col in df_station.columns]

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, col in enumerate(air_quality_vars):
    if idx < len(axes):
        ax = axes[idx]
        df_station.boxplot(column=col, ax=ax)
        ax.set_title(f'{col} Distribution', fontsize=12, fontweight='bold')
        ax.set_ylabel('Value', fontsize=10)
        
        # Tính IQR và outliers
        Q1 = df_station[col].quantile(0.25)
        Q3 = df_station[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df_station[col] < Q1 - 1.5*IQR) | (df_station[col] > Q3 + 1.5*IQR)).sum()
        outlier_pct = outliers / len(df_station) * 100
        
        ax.text(0.5, 0.95, f'Outliers: {outliers} ({outlier_pct:.2f}%)', 
                transform=ax.transAxes, ha='center', va='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

In [ ]:
# Phân tích skewness (độ lệch)
print("\n=" * 60)
print("PHÂN TÍCH ĐỘ LỆCH (SKEWNESS)")
print("=" * 60)

skewness = df_station[numeric_cols].skew().sort_values(ascending=False)
print(skewness)

print("\n⚠️ NHẬN XÉT:")
highly_skewed = skewness[abs(skewness) > 1]
if len(highly_skewed) > 0:
    print(f"Các biến có độ lệch cao (|skew| > 1): {list(highly_skewed.index)}")
    print("→ Cần xem xét log-transform hoặc phương pháp chuẩn hóa khác")
else:
    print("✅ Không có biến nào có độ lệch quá cao")

## 6. Vẽ chuỗi PM2.5 theo thời gian

**Mục tiêu:** Visualize chuỗi PM2.5 - toàn bộ giai đoạn + zoom vào 1-2 tháng

In [ ]:
# Loại bỏ missing values để vẽ
df_plot = df_station[['datetime', 'PM2.5']].dropna()

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Plot 1: Toàn bộ chuỗi thời gian
ax1 = axes[0]
ax1.plot(df_plot['datetime'], df_plot['PM2.5'], linewidth=0.8, alpha=0.7, color='steelblue')
ax1.set_title('PM2.5 Time Series - Toàn bộ giai đoạn', fontsize=14, fontweight='bold')
ax1.set_ylabel('PM2.5 (μg/m³)', fontsize=11)
ax1.grid(True, alpha=0.3)

# Thêm moving average
df_plot['MA_7d'] = df_plot['PM2.5'].rolling(window=24*7, center=True).mean()
ax1.plot(df_plot['datetime'], df_plot['MA_7d'], linewidth=2, color='red', 
         label='7-day moving avg', alpha=0.8)
ax1.legend()

# Plot 2: Zoom vào 1 tháng (ví dụ: tháng 1/2017 - giai đoạn cuối)
ax2 = axes[1]
zoom1_start = '2017-01-01'
zoom1_end = '2017-02-01'
df_zoom1 = df_plot[(df_plot['datetime'] >= zoom1_start) & (df_plot['datetime'] < zoom1_end)]
ax2.plot(df_zoom1['datetime'], df_zoom1['PM2.5'], linewidth=1.5, color='green', marker='o', markersize=2)
ax2.set_title(f'PM2.5 - Zoom vào tháng 1/2017', fontsize=13, fontweight='bold')
ax2.set_ylabel('PM2.5 (μg/m³)', fontsize=11)
ax2.grid(True, alpha=0.3)

# Plot 3: Zoom vào 1 tháng khác (ví dụ: tháng 12/2013 - giai đoạn đầu)
ax3 = axes[2]
zoom2_start = '2013-12-01'
zoom2_end = '2014-01-01'
df_zoom2 = df_plot[(df_plot['datetime'] >= zoom2_start) & (df_plot['datetime'] < zoom2_end)]
ax3.plot(df_zoom2['datetime'], df_zoom2['PM2.5'], linewidth=1.5, color='purple', marker='o', markersize=2)
ax3.set_title(f'PM2.5 - Zoom vào tháng 12/2013', fontsize=13, fontweight='bold')
ax3.set_ylabel('PM2.5 (μg/m³)', fontsize=11)
ax3.set_xlabel('Time', fontsize=11)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
print("- Quan sát xu hướng, tính mùa vụ, và các spike bất thường")
print("- Zoom vào các tháng cụ thể để thấy dao động hàng ngày/hàng tuần")

## 7. Kiểm tra Tự Tương Quan (Autocorrelation)

**Mục tiêu:** So sánh tương quan của PM2.5 với lag 24h (chu kỳ ngày) và lag 168h (chu kỳ tuần)

In [ ]:
# Tạo lagged variables
df_corr = df_station[['datetime', 'PM2.5']].dropna().copy()
df_corr['PM25_lag1'] = df_corr['PM2.5'].shift(1)
df_corr['PM25_lag24'] = df_corr['PM2.5'].shift(24)  # 24 giờ
df_corr['PM25_lag168'] = df_corr['PM2.5'].shift(168)  # 168 giờ = 1 tuần

# Tính correlation
print("=" * 60)
print("CORRELATION ANALYSIS - PM2.5 vs LAGGED VALUES")
print("=" * 60)

corr_lag1 = df_corr['PM2.5'].corr(df_corr['PM25_lag1'])
corr_lag24 = df_corr['PM2.5'].corr(df_corr['PM25_lag24'])
corr_lag168 = df_corr['PM2.5'].corr(df_corr['PM25_lag168'])

print(f"Correlation PM2.5(t) vs PM2.5(t-1):   {corr_lag1:.4f}")
print(f"Correlation PM2.5(t) vs PM2.5(t-24):  {corr_lag24:.4f}  ← Chu kỳ NGÀY")
print(f"Correlation PM2.5(t) vs PM2.5(t-168): {corr_lag168:.4f}  ← Chu kỳ TUẦN")

# Visualize ACF and PACF
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# ACF
plot_acf(df_corr['PM2.5'].dropna(), lags=200, ax=axes[0])
axes[0].set_title('Autocorrelation Function (ACF) - PM2.5', fontsize=13, fontweight='bold')
axes[0].axvline(x=24, color='red', linestyle='--', alpha=0.7, label='Lag 24h (daily)')
axes[0].axvline(x=168, color='green', linestyle='--', alpha=0.7, label='Lag 168h (weekly)')
axes[0].legend()

# PACF
plot_pacf(df_corr['PM2.5'].dropna(), lags=50, ax=axes[1])
axes[1].set_title('Partial Autocorrelation Function (PACF) - PM2.5', fontsize=13, fontweight='bold')
axes[1].axvline(x=24, color='red', linestyle='--', alpha=0.7, label='Lag 24h (daily)')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
if corr_lag24 > 0.3:
    print(f"✅ Có chu kỳ NGÀY rõ ràng (corr={corr_lag24:.3f}) → Lag 24h rất có ý nghĩa cho mô hình")
if corr_lag168 > 0.2:
    print(f"✅ Có chu kỳ TUẦN (corr={corr_lag168:.3f}) → Có thể thêm lag 168h")
else:
    print(f"⚠️ Chu kỳ tuần không rõ ràng (corr={corr_lag168:.3f})")

## 8. Kiểm tra Tính Dừng (Stationarity Test)

**Mục tiêu:** Sử dụng ADF và KPSS test để kiểm tra tính dừng của chuỗi PM2.5

In [ ]:
# Chuẩn bị data (loại bỏ missing)
pm25_series = df_station['PM2.5'].dropna()

print("=" * 60)
print("KIỂM ĐỊNH TÍNH DỪNG - STATIONARITY TESTS")
print("=" * 60)

# ADF Test (Augmented Dickey-Fuller)
print("\n1. AUGMENTED DICKEY-FULLER (ADF) TEST")
print("-" * 60)
adf_result = adfuller(pm25_series, autolag='AIC')
print(f"ADF Statistic: {adf_result[0]:.6f}")
print(f"P-value: {adf_result[1]:.6f}")
print(f"Critical Values:")
for key, value in adf_result[4].items():
    print(f"  {key}: {value:.3f}")

if adf_result[1] < 0.05:
    print("\n✅ KẾT LUẬN ADF: Chuỗi DỪNG (reject H0, p < 0.05)")
    adf_conclusion = "stationary"
else:
    print("\n❌ KẾT LUẬN ADF: Chuỗi KHÔNG DỪNG (fail to reject H0, p >= 0.05)")
    adf_conclusion = "non-stationary"

# KPSS Test
print("\n2. KWIATKOWSKI-PHILLIPS-SCHMIDT-SHIN (KPSS) TEST")
print("-" * 60)
kpss_result = kpss(pm25_series, regression='c', nlags='auto')
print(f"KPSS Statistic: {kpss_result[0]:.6f}")
print(f"P-value: {kpss_result[1]:.6f}")
print(f"Critical Values:")
for key, value in kpss_result[3].items():
    print(f"  {key}: {value:.3f}")

if kpss_result[1] < 0.05:
    print("\n❌ KẾT LUẬN KPSS: Chuỗi KHÔNG DỪNG (reject H0, p < 0.05)")
    kpss_conclusion = "non-stationary"
else:
    print("\n✅ KẾT LUẬN KPSS: Chuỗi DỪNG (fail to reject H0, p >= 0.05)")
    kpss_conclusion = "stationary"

# Tổng hợp kết luận
print("\n" + "=" * 60)
print("TỔNG HỢP KẾT LUẬN")
print("=" * 60)
print(f"ADF Test:  {adf_conclusion.upper()}")
print(f"KPSS Test: {kpss_conclusion.upper()}")

if adf_conclusion == "stationary" and kpss_conclusion == "stationary":
    print("\n✅✅ CẢ HAI TEST ĐỒNG Ý: Chuỗi PM2.5 là DỪNG")
    print("→ Có thể dùng trực tiếp cho ARIMA với d=0")
elif adf_conclusion == "non-stationary" and kpss_conclusion == "non-stationary":
    print("\n❌❌ CẢ HAI TEST ĐỒNG Ý: Chuỗi PM2.5 KHÔNG DỪNG")
    print("→ Cần differencing (d>=1) trước khi dùng ARIMA")
else:
    print("\n⚠️ HAI TEST MÂU THUẪN - cần xem xét kỹ hơn")
    print("→ Thử differencing và kiểm tra lại, hoặc xem xét xu hướng/seasonality")

## 9. Kiểm tra Differencing

Nếu chuỗi không dừng, thử differencing và kiểm tra lại

In [ ]:
# First difference
pm25_diff1 = pm25_series.diff().dropna()

print("=" * 60)
print("KIỂM ĐỊNH SAU FIRST DIFFERENCING")
print("=" * 60)

# ADF on differenced series
adf_diff1 = adfuller(pm25_diff1, autolag='AIC')
print(f"\nADF Test (after diff):")
print(f"  Statistic: {adf_diff1[0]:.6f}")
print(f"  P-value: {adf_diff1[1]:.6f}")

if adf_diff1[1] < 0.05:
    print("  ✅ Chuỗi SAU DIFFERENCING là DỪNG")
    print("  → Nên dùng d=1 cho ARIMA")
else:
    print("  ❌ Vẫn chưa dừng, có thể cần d=2")

# Visualize original vs differenced
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Original
axes[0].plot(pm25_series.values[:1000], linewidth=1)
axes[0].set_title('PM2.5 Original Series (first 1000 points)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('PM2.5')
axes[0].grid(True, alpha=0.3)

# Differenced
axes[1].plot(pm25_diff1.values[:1000], linewidth=1, color='orange')
axes[1].set_title('PM2.5 After First Differencing (first 1000 points)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Δ PM2.5')
axes[1].set_xlabel('Time index')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 10. Phân tích: Biến thiếu nào đáng lo nhất cho dự báo PM2.5?

**Câu hỏi:** Thiếu theo biến nào là đáng lo nhất cho dự báo PM2.5 và vì sao?

In [ ]:
print("=" * 60)
print("PHÂN TÍCH: BIẾN THIẾU ĐÁNG LO NHẤT CHO DỰ BÁO PM2.5")
print("=" * 60)

# Tính correlation của các biến với PM2.5
feature_cols = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
feature_cols = [col for col in feature_cols if col in df_station.columns]

correlations = df_station[feature_cols + ['PM2.5']].corr()['PM2.5'].drop('PM2.5').abs().sort_values(ascending=False)
missing_rates = df_station[feature_cols].isna().mean() * 100

# Tạo bảng so sánh
comparison = pd.DataFrame({
    'Correlation_with_PM25': correlations,
    'Missing_Rate_%': missing_rates,
    'Impact_Score': correlations * (1 - missing_rates/100)  # High correlation * low missing = good
}).sort_values('Correlation_with_PM25', ascending=False)

print("\nBẢNG SO SÁNH:")
print(comparison)

# Tìm biến đáng lo nhất: high correlation + high missing
comparison['Worry_Score'] = correlations * (missing_rates/100)  # High corr * high missing = worry
most_worrying = comparison.sort_values('Worry_Score', ascending=False).head(3)

print("\n" + "=" * 60)
print("BIẾN ĐÁNG LO NHẤT (correlation cao + missing nhiều):")
print("=" * 60)
print(most_worrying[['Correlation_with_PM25', 'Missing_Rate_%', 'Worry_Score']])

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(comparison['Missing_Rate_%'], comparison['Correlation_with_PM25'], 
                     s=200, alpha=0.6, c=comparison['Worry_Score'], cmap='Reds')
ax.set_xlabel('Missing Rate (%)', fontsize=12)
ax.set_ylabel('|Correlation with PM2.5|', fontsize=12)
ax.set_title('Biến nào đáng lo? (góc phải trên = đáng lo nhất)', fontsize=13, fontweight='bold')

# Annotate points
for idx, row in comparison.iterrows():
    ax.annotate(idx, (row['Missing_Rate_%'], row['Correlation_with_PM25']), 
                fontsize=9, ha='right')

plt.colorbar(scatter, label='Worry Score')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n" + "=" * 60)
print("KẾT LUẬN")
print("=" * 60)
top_worry = most_worrying.index[0]
print(f"\n🔴 BIẾN ĐÁNG LO NHẤT: {top_worry}")
print(f"   - Correlation với PM2.5: {comparison.loc[top_worry, 'Correlation_with_PM25']:.3f}")
print(f"   - Tỷ lệ thiếu: {comparison.loc[top_worry, 'Missing_Rate_%']:.2f}%")
print(f"\nVÌ SAO?")
print(f"   → Biến này có tương quan CAO với PM2.5, nghĩa là rất quan trọng cho dự báo")
print(f"   → Nhưng lại bị thiếu NHIỀU, làm giảm khả năng sử dụng thông tin này")
print(f"   → Cần xử lý missing cẩn thận (imputation hoặc loại bỏ) trước khi training")

print(f"\n🟡 BIẾN QUAN TRỌNG NHƯNG ÍT THIẾU:")
good_features = comparison[(comparison['Correlation_with_PM25'] > 0.3) & 
                           (comparison['Missing_Rate_%'] < 5)]
if len(good_features) > 0:
    print(list(good_features.index))
    print("   → Đây là các biến TỐT để dùng cho mô hình")
else:
    print("   Không có biến nào đủ tốt (corr > 0.3 và missing < 5%)")

## 📝 TÓM TẮT FINDINGS

### Câu trả lời cho Q1:

1. **Khoảng thời gian & tần suất:** 
   - ✅ Đã kiểm tra start/end date và coverage
   - ✅ Xác nhận tần suất theo giờ và tính liên tục

2. **Tỷ lệ thiếu:**
   - ✅ Tính missing rate cho từng biến
   - ✅ Phân tích missing pattern theo thời gian (tập trung hay rải đều)

3. **Outliers & Distribution:**
   - ✅ Boxplot cho tất cả biến quan trọng
   - ✅ Quantile analysis
   - ✅ Skewness analysis

4. **PM2.5 Time Series:**
   - ✅ Toàn bộ giai đoạn với moving average
   - ✅ Zoom vào 2 tháng khác nhau để thấy rõ pattern

5. **Tự tương quan:**
   - ✅ Correlation với lag 24h (chu kỳ ngày)
   - ✅ Correlation với lag 168h (chu kỳ tuần)
   - ✅ ACF/PACF plots

6. **Tính dừng:**
   - ✅ ADF test
   - ✅ KPSS test
   - ✅ Kiểm tra differencing

7. **Biến thiếu đáng lo:**
   - ✅ Phân tích correlation vs missing rate
   - ✅ Xác định biến nào quan trọng nhưng thiếu nhiều
   - ✅ Giải thích vì sao đáng lo